# Matrix Factorization

The first model to compare with baseline is matrix factorization. Since we need to train implicit data, we consider Alternating Least Squares (ALS) as MF model which simple and truely competitive.

In [20]:
import pickle

import numpy as np
import pandas as pd

from implicit.als import AlternatingLeastSquares
from scipy import sparse
from tqdm.auto import tqdm

## Load data

In [4]:
train = pd.read_parquet("./data/train.parquet")
valid = pd.read_parquet("./data/valid.parquet")
test = pd.read_parquet("./data/test.parquet")

In [12]:
with open("./data/user_mapper.dict", "rb") as f:
    user_mapper =  pickle.load(f)
with open("./data/item_mapper.dict", "rb") as f:
    item_mapper =  pickle.load(f)

In [14]:
train_mat = np.zeros((len(user_mapper), len(item_mapper)))
valid_mat = np.zeros((len(user_mapper), len(item_mapper)))
test_mat = np.zeros((len(user_mapper), len(item_mapper)))

In [18]:
for _, user, item, rating in tqdm(train.itertuples(), total=len(train)):
    train_mat[user, item] = 1

for _, user, item, rating in tqdm(valid.itertuples(), total=len(valid)):
    valid_mat[user, item] = 1

for _, user, item, rating in tqdm(test.itertuples(), total=len(test)):
    test_mat[user, item] = 1

  0%|          | 0/1022381 [00:00<?, ?it/s]

  0%|          | 0/220993 [00:00<?, ?it/s]

  0%|          | 0/222265 [00:00<?, ?it/s]

In [22]:
train_csr = sparse.csr_matrix(train_mat)
valid_csr = sparse.csr_matrix(valid_mat)
test_csr = sparse.csr_matrix(test_mat)

## Modeling

In [23]:
alpha = 40.

train_csr = train_csr * alpha
valid_csr = valid_csr * alpha

In [92]:
model = AlternatingLeastSquares(
    factors=64,
    regularization=0.01,
    iterations=1,
    random_state=22,
)

In [ ]:
model.fit(train_csr)

  0%|          | 0/15 [00:00<?, ?it/s]

In [91]:
model.recommend(0, train_csr[0], N=32, filter_already_liked_items=True)

(array([32273, 20069, 22092,  5457, 40038,  3133, 25945, 24425,  6567,
         5079, 45237, 10144, 51855, 32348, 29328, 48594, 48637,   245,
        12768, 12866,  7643, 25070,   565, 45804, 45841,  2848, 18249,
        23738, 24671,  6435, 38337, 44728], dtype=int32),
 array([1.2044903 , 1.101104  , 1.0891685 , 1.0588266 , 1.0465364 ,
        1.0130321 , 1.0114012 , 1.0112301 , 1.0091028 , 1.0066841 ,
        0.99797326, 0.9848322 , 0.98256505, 0.9741199 , 0.9733227 ,
        0.9618347 , 0.96148384, 0.95805734, 0.94813496, 0.94772613,
        0.9436702 , 0.9376612 , 0.93549174, 0.9280097 , 0.91752446,
        0.9174148 , 0.9172937 , 0.9143025 , 0.91160214, 0.9112837 ,
        0.90554917, 0.9048336 ], dtype=float32))

```
(array([32273, 30131,  3133, 40038,  7141,  1684, 20069, 27267, 24561,
        19627, 11124, 34856, 25945, 45052, 15750, 46222, 20147,  6850,
        38321,  6751, 18253, 45436,   628, 49855, 14101,  7643, 35249,
        28894, 25439, 34119, 52706, 26291], dtype=int32),
 array([0.84358174, 0.812567  , 0.7517934 , 0.7254622 , 0.7208959 ,
        0.72008276, 0.7066336 , 0.7026261 , 0.6762129 , 0.6734191 ,
        0.67070127, 0.6703811 , 0.6582423 , 0.6354214 , 0.63338333,
        0.62986785, 0.6207571 , 0.6168754 , 0.6130548 , 0.60986245,
        0.60888875, 0.6077634 , 0.6017588 , 0.60080343, 0.5926179 ,
        0.5856929 , 0.58440983, 0.5822266 , 0.57806206, 0.5760103 ,
        0.5744134 , 0.5718856 ], dtype=float32))
```

In [29]:
model.iterations

1

In [28]:
model.fit(train_csr)

  0%|          | 0/1 [00:00<?, ?it/s]